# 🚀 PhoBERT Email Classifier Training

**VeryGoodMail - Email Classification System**

Notebook này hướng dẫn cách train PhoBERT để:
1. **Spam Detection**: Phân loại email spam/ham
2. **Sentiment Analysis**: Phân tích cảm xúc email (tích cực/tiêu cực/trung lập)
3. **Email Categorization**: Phân loại vào thư mục (Quan trọng, Xã hội, Khuyến mãi, Cập nhật)

---
© 2025 VeryGoodMail by **Hoàn**

## 📋 Bước 1: Cài đặt thư viện

In [ ]:
!pip install -q transformers datasets torch accelerate sentencepiece
!pip install -q underthesea  # Vietnamese NLP toolkit
!pip install -q scikit-learn pandas numpy

In [ ]:
import os
import json
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 📊 Bước 2: Chuẩn bị dữ liệu training

### Tùy chọn A: Sử dụng dữ liệu mẫu (demo)
### Tùy chọn B: Upload dữ liệu của bạn

In [ ]:
# ============================================================
# Dữ liệu mẫu cho demo - Trong thực tế cần nhiều dữ liệu hơn!
# ============================================================

# Spam Detection Dataset
spam_data = {
    'text': [
        # Spam samples (Vietnamese)
        "Chúc mừng! Bạn đã trúng thưởng 1 tỷ đồng. Nhấn vào đây để nhận ngay",
        "Kiếm tiền online dễ dàng, thu nhập 50 triệu/tháng không cần vốn",
        "Giảm cân nhanh chóng 10kg trong 1 tuần, cam kết hiệu quả 100%",
        "Khuyến mãi sốc! Chỉ hôm nay, mua 1 tặng 10, nhanh tay đặt hàng",
        "Bạn được chọn ngẫu nhiên để nhận iPhone 15 Pro Max miễn phí",
        "Vay tiền online không cần thế chấp, duyệt nhanh trong 5 phút",
        "Thuốc tăng cường sinh lý nam giới, hiệu quả sau 1 lần dùng",
        "Click ngay để nhận 500.000đ vào tài khoản của bạn",
        "Bạn có khoản vay chưa thanh toán, liên hệ ngay để tránh bị khóa",
        "Chương trình khách hàng thân thiết, bạn nhận được 10 triệu điểm thưởng",
        
        # Spam samples (English)
        "Congratulations! You have won $1,000,000 in our lottery",
        "Make money fast from home, earn $10,000 per week",
        "Click here to claim your free iPhone 15 now",
        "Nigerian prince needs your help to transfer $50 million",
        "Hot singles in your area want to meet you tonight",
        "Buy cheap medications online without prescription",
        "Your account has been compromised, click to verify immediately",
        "Limited time offer! 90% discount on all luxury watches",
        "You are selected for exclusive investment opportunity",
        "Work from home and earn passive income easily",
        
        # Ham samples (Vietnamese)
        "Anh ơi, cuộc họp ngày mai lúc 9h sáng tại phòng họp A nhé",
        "Em gửi anh báo cáo tháng 11, anh review giúp em với ạ",
        "Cảm ơn anh đã hỗ trợ dự án, em rất trân trọng",
        "Nhắc lịch: Deadline nộp bài vào thứ 6 tuần này",
        "Đây là biên bản cuộc họp hôm qua, mời mọi người xem",
        "Anh cho em hỏi về tiến độ dự án XYZ được không ạ?",
        "Xác nhận: Em đã nhận được tài liệu anh gửi",
        "Mời anh/chị tham dự buổi workshop vào thứ 3 tuần sau",
        "Gửi anh bảng chấm công tháng 11, anh ký duyệt giúp em",
        "Team mình họp standup lúc 10h sáng mai nhé",
        
        # Ham samples (English)
        "Meeting scheduled for tomorrow at 10 AM in room 301",
        "Please review the attached quarterly report",
        "Thank you for your help with the project presentation",
        "Reminder: Deadline for submission is this Friday",
        "Here are the meeting notes from yesterday's discussion",
        "Can you provide an update on the project status?",
        "Confirming receipt of the documents you sent",
        "You are invited to the workshop next Tuesday",
        "Monthly timesheet attached for your approval",
        "Team standup tomorrow at 9 AM as usual",
    ],
    'label': [1]*10 + [1]*10 + [0]*10 + [0]*10  # 1 = spam, 0 = ham
}

# Sentiment Dataset
sentiment_data = {
    'text': [
        # Positive (Vietnamese)
        "Cảm ơn anh rất nhiều, dịch vụ tuyệt vời!",
        "Em rất hài lòng với sản phẩm này, chất lượng xuất sắc",
        "Đội ngũ hỗ trợ rất nhiệt tình và chuyên nghiệp",
        "Tuyệt vời! Đúng như mong đợi, sẽ ủng hộ tiếp",
        "Giao hàng nhanh, đóng gói cẩn thận, rất đẹp",
        
        # Positive (English)
        "Thank you so much, excellent service!",
        "Very satisfied with this product, outstanding quality",
        "The support team is very helpful and professional",
        "Wonderful! Exactly as expected, will continue to support",
        "Fast delivery, careful packaging, very nice",
        
        # Negative (Vietnamese)
        "Thất vọng quá, sản phẩm kém chất lượng",
        "Dịch vụ tệ, nhân viên thiếu chuyên nghiệp",
        "Đợi mãi không thấy phản hồi, rất bực mình",
        "Hàng giao sai, không đúng mô tả, yêu cầu hoàn tiền",
        "Quá tệ! Không bao giờ mua ở đây nữa",
        
        # Negative (English)
        "Very disappointed, poor quality product",
        "Terrible service, unprofessional staff",
        "Been waiting forever, no response, very frustrated",
        "Wrong item delivered, not as described, requesting refund",
        "Awful! Never buying from here again",
        
        # Neutral (Vietnamese)
        "Em xác nhận đã nhận được email của anh",
        "Đây là báo cáo tuần này, mời anh xem",
        "Thông báo: Hệ thống bảo trì vào 10h tối nay",
        "Gửi anh file đính kèm theo yêu cầu",
        "Nhắc lịch: Cuộc họp lúc 2h chiều mai",
        
        # Neutral (English)
        "Confirming receipt of your email",
        "Here is this week's report for your review",
        "Notice: System maintenance at 10 PM tonight",
        "Attached file as requested",
        "Reminder: Meeting at 2 PM tomorrow",
    ],
    'label': [2]*5 + [2]*5 + [0]*5 + [0]*5 + [1]*5 + [1]*5  # 0=negative, 1=neutral, 2=positive
}

# Category Dataset
category_data = {
    'text': [
        # Important
        "KHẨN CẤP: Cần xử lý ngay trước 5h chiều hôm nay",
        "Quan trọng: Yêu cầu phê duyệt gấp từ CEO",
        "URGENT: Action required before end of day",
        "Important: Your password will expire in 24 hours",
        "Cảnh báo bảo mật: Phát hiện đăng nhập bất thường",
        
        # Social
        "Nguyễn Văn A đã thích bài viết của bạn",
        "Bạn có 5 yêu cầu kết bạn mới",
        "John commented on your photo",
        "You have 3 new followers on Instagram",
        "Nhắc nhở: Sinh nhật của Minh vào ngày mai",
        
        # Promotions
        "Giảm giá 50% tất cả sản phẩm mùa hè",
        "Flash sale: Chỉ 24 giờ, giảm đến 70%",
        "Summer sale: 50% off all items",
        "New collection available now - Shop today!",
        "Ưu đãi độc quyền dành riêng cho bạn",
        
        # Updates
        "Đơn hàng #12345 của bạn đã được giao thành công",
        "Cập nhật: Gói hàng đang trên đường giao",
        "Your order has been shipped",
        "Software update available for your device",
        "Sao kê tài khoản tháng 11 đã sẵn sàng",
        
        # Primary (general)
        "Cuộc họp team vào thứ 2 tuần sau",
        "Gửi anh báo cáo dự án như đã trao đổi",
        "Team meeting scheduled for next Monday",
        "Project update as discussed yesterday",
        "Xin chào, em muốn hỏi về vấn đề...",
    ],
    'label': [0]*5 + [1]*5 + [2]*5 + [3]*5 + [4]*5  # 0=important, 1=social, 2=promotions, 3=updates, 4=primary
}

print(f"Spam dataset: {len(spam_data['text'])} samples")
print(f"Sentiment dataset: {len(sentiment_data['text'])} samples")
print(f"Category dataset: {len(category_data['text'])} samples")

## 🔧 Bước 3: Tạo Dataset class và load PhoBERT

In [ ]:
class EmailDataset(Dataset):
    """Custom Dataset for email classification."""
    
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load PhoBERT tokenizer
print("Loading PhoBERT tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
print("Tokenizer loaded successfully!")

## 🏋️ Bước 4: Training Functions

In [ ]:
def compute_metrics(pred):
    """Compute metrics for evaluation."""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def train_classifier(train_data, num_labels, model_name, output_dir, epochs=5):
    """
    Train a PhoBERT classifier.
    
    Args:
        train_data: dict with 'text' and 'label' keys
        num_labels: number of classification labels
        model_name: name for saving the model
        output_dir: directory to save the model
        epochs: number of training epochs
    """
    print(f"\n{'='*50}")
    print(f"Training: {model_name}")
    print(f"{'='*50}")
    
    # Split data
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        train_data['text'], train_data['label'],
        test_size=0.2, random_state=SEED, stratify=train_data['label']
    )
    
    print(f"Train samples: {len(train_texts)}")
    print(f"Validation samples: {len(val_texts)}")
    
    # Create datasets
    train_dataset = EmailDataset(train_texts, train_labels, tokenizer)
    val_dataset = EmailDataset(val_texts, val_labels, tokenizer)
    
    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(
        "vinai/phobert-base",
        num_labels=num_labels
    )
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./{output_dir}",
        num_train_epochs=epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
    )
    
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    # Train
    print("\nStarting training...")
    trainer.train()
    
    # Evaluate
    print("\nEvaluation results:")
    eval_results = trainer.evaluate()
    for key, value in eval_results.items():
        print(f"  {key}: {value:.4f}")
    
    # Save model
    final_path = f"./models/{model_name}"
    trainer.save_model(final_path)
    tokenizer.save_pretrained(final_path)
    print(f"\nModel saved to: {final_path}")
    
    return trainer, eval_results

## 🚀 Bước 5: Train các models

In [ ]:
# Create models directory
os.makedirs('./models', exist_ok=True)

# Train Spam Classifier
spam_trainer, spam_results = train_classifier(
    train_data=spam_data,
    num_labels=2,
    model_name="spam_classifier",
    output_dir="spam_output",
    epochs=5
)

In [ ]:
# Train Sentiment Classifier
sentiment_trainer, sentiment_results = train_classifier(
    train_data=sentiment_data,
    num_labels=3,
    model_name="sentiment_classifier",
    output_dir="sentiment_output",
    epochs=5
)

In [ ]:
# Train Category Classifier
category_trainer, category_results = train_classifier(
    train_data=category_data,
    num_labels=5,
    model_name="category_classifier",
    output_dir="category_output",
    epochs=5
)

## 🧪 Bước 6: Test models

In [ ]:
def test_classifier(model_path, test_texts, label_names):
    """Test a trained classifier with sample texts."""
    print(f"\nTesting model: {model_path}")
    print("-" * 50)
    
    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    model.to(device)
    model.eval()
    
    for text in test_texts:
        inputs = tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=256,
            return_tensors='pt'
        ).to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            pred_idx = probs.argmax().item()
            confidence = probs[0][pred_idx].item()
        
        print(f"\nText: {text[:80]}..." if len(text) > 80 else f"\nText: {text}")
        print(f"Prediction: {label_names[pred_idx]} ({confidence:.2%})")

# Test Spam Classifier
spam_test_texts = [
    "Bạn đã trúng thưởng 100 triệu đồng! Click ngay để nhận",
    "Cuộc họp team vào 9h sáng mai tại phòng họp A",
    "URGENT: You have won a free cruise vacation!",
    "Please review the attached quarterly report"
]
test_classifier("./models/spam_classifier", spam_test_texts, ["Ham", "Spam"])

# Test Sentiment Classifier
sentiment_test_texts = [
    "Cảm ơn anh rất nhiều, dịch vụ tuyệt vời!",
    "Thất vọng quá, sản phẩm kém chất lượng",
    "Đây là báo cáo tuần này, mời anh xem",
]
test_classifier("./models/sentiment_classifier", sentiment_test_texts, ["Negative", "Neutral", "Positive"])

## 📦 Bước 7: Export models

In [ ]:
# Zip models for download
!zip -r phobert_email_models.zip ./models/

# Download link (in Colab)
from google.colab import files
files.download('phobert_email_models.zip')

## 📖 Hướng dẫn sử dụng models

### 1. Giải nén models
```bash
unzip phobert_email_models.zip -d /path/to/phobert-service/
```

### 2. Chạy PhoBERT service
```bash
cd Email-System-Server/phobert-service
pip install -r requirements.txt
USE_PHOBERT=true python main.py
```

### 3. Test API
```bash
curl -X POST http://localhost:8000/classify \
  -H "Content-Type: application/json" \
  -d '{"subject": "Test email", "body": "Hello world"}'
```

---

## 💡 Tips để cải thiện accuracy

1. **Thêm nhiều dữ liệu**: Dataset hiện tại chỉ có ~30-40 samples mỗi loại. Cần ít nhất 1000+ samples để có kết quả tốt.

2. **Data augmentation**: Sử dụng các kỹ thuật như:
   - Back-translation (dịch sang ngôn ngữ khác rồi dịch lại)
   - Synonym replacement
   - Random insertion/deletion

3. **Hyperparameter tuning**: Thử các giá trị khác nhau cho learning rate, batch size, epochs.

4. **Cross-validation**: Sử dụng k-fold cross-validation để đánh giá model chính xác hơn.

5. **Ensemble**: Kết hợp nhiều models để có kết quả tốt hơn.

---

© 2025 VeryGoodMail by **Hoàn**